## LSTM Layer 파헤치기
Sequence 혹은 Time Series (시계열) 데이터를 다룰 때, LSTM layer를 어떻게 활용하여 접근하면 되는지 이해하기 위한 튜토리얼 코드입니다.

### 필요한 모듈을 import 합니다.

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model

### 0. Hyper Parameter 설명
출처: keras.io

- units: 양의 정수, 아웃풋 공간의 차원입니다.
- activation: 사용할 활성화 함수 (활성화를 참조하십시오). 디폴트: 쌍곡 탄젠트 (tanh). None을 전달하는 경우, 활성화가 적용되지 않습니다 (다시 말해, "선형적" 활성화: a(x) = x).
- recurrent_activation: 순환 단계에 사용할 활성화 함수 (활성화를 참조하십시오). 디폴트 값: 하드 시그모이드 (hard_sigmoid). None을 전달하는 경우, 활성화가 적용되지 않습니다 (다시 말해, "선형적" 활성화: a(x) = x).
- use_bias: 불리언, 레이어가 편향 벡터를 사용하는지 여부.
- kernel_initializer: kernel 가중치 행렬의 초기값 설정기. 인풋의 선형적 변형에 사용됩니다 ( 초기값 설정기를 참조하십시오).
- recurrent_initializer: recurrent_kernel 가중치 행렬의 초기값 설정기. 순환 상태의 선형적 변형에 사용됩니다 ( 초기값 설정기를 참조하십시오).
- bias_initializer: 편향 벡터의 초기값 설정기 ( 초기값 설정기를 참조하십시오).
- unit_forget_bias: 불리언. 참일 경우, 초기값 설정 시 망각 회로에 1을 더합니다. 참으로 설정 시 강제적으로 bias_initializer="zeros"가 됩니다. 이는 [Jozefowicz et al. (2015)](http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf) 에서 권장됩니다. .
- kernel_regularizer: kernel 가중치 행렬에 적용되는 정규화 함수 (정규화를 참조하십시오).
- recurrent_regularizer: recurrent_kernel 가중치 행렬에 적용되는 정규화 함수 (정규화를 참조하십시오).
- bias_regularizer: 편향 벡터에 적용되는 정규화 함수 (정규화를 참조하십시오).
- activity_regularizer: 레이어의 아웃풋(레이어의 “활성화”)에 적용되는 정규화 함수 (정규화를 참조하십시오).
- kernel_constraint: kernel 가중치 행렬에 적용되는 제약 함수 (제약을 참조하십시오).
- recurrent_constraint: recurrent_kernel 가중치 행렬에 적용되는 제약 함수 (제약을 참조하십시오).
- bias_constraint: 편향 벡터에 적용하는 제약 함수 (제약을 참조하십시오).
- dropout: 0과 1사이 부동소수점. 인풋의 선형적 변형을 실행하는데 드롭시킬(고려하지 않을) 유닛의 비율.
- recurrent_dropout: 0과 1사이 부동소수점. 순환 상태의 선형적 변형을 실행하는데 드롭시킬(고려하지 않을) 유닛의 비율.
- implementation: 실행 모드, 1 혹은 2. 모드 1은 비교적 많은 수의 소규모의 점곱과 덧셈을 이용해 연산을 구성하는데 반해, 모드 2는 이를 소수의 대규모 연산으로 묶습니다. 이 두 모드는, 하드웨어나 어플리케이션에 따라서 성능의 차이를 보입니다.
- return_sequences: 불리언. 아웃풋 시퀀스의 마지막 아웃풋을 반환할지, 혹은 시퀀스 전체를 반환할지 여부.
- return_state: 불리언. 아웃풋에 더해 마지막 상태도 반환할지 여부. 상태 리스트의 반환된 성분은 각각 은닉 성분과 셀 상태입니다.
- go_backwards: 불리언 (디폴트 값은 거짓). 참인 경우, 인풋 시퀀스를 거꾸로 처리하여 뒤집힌 시퀀스를 반환합니다.
- stateful: 불리언 (디폴트 값은 거짓). 참인 경우, 배치 내 색인 i의 각 샘플의 마지막 상태가 다음 배치의 색인 i 샘플의 초기 상태로 사용됩니다.
- unroll: 불리언 (디폴트 값은 거짓). 참인 경우, 신경망을 펼쳐서 사용하고 그렇지 않은 경우 심볼릭 루프가 사용됩니다. 신경망을 펼쳐 순환 신경망의 속도를 높일 수 있지만, 메모리 소모가 큰 경향이 있습니다. 신경망 펼치기는 짧은 시퀀스에만 적합합니다.

### 1. LSTM Layer와 input_shape
주요 hyper parameter는 다음과 같습니다.

- batch: 32
- time_step: 3
- window_size: 25

#### sample data 생성
아래와 같이 샘플 데이터를 생성합니다.

In [2]:
x = tf.random.uniform(shape=(32, 25, 1))

In [3]:
x.shape

TensorShape([32, 25, 1])

In [ ]:
32는 batch의 크기, 25는 time_step의 크기, 1은 feature의 갯수를 나타냅니다.

여기서 batch는 얼마만큼 batch로 묶어 주느냐에 따라 달라지는 hyper parameter이므로 크게 걱정할 이유가 없습니다.

25는 window_size를 나타내며, 일자로 예를 들자면, 25일치의 time_step을 input으로 공급하겠다는 겁니다.

1은 feature_size이며, 주가 데이터를 예로 들자면, 종가 데이터 한 개만 본다면 1로 설정합니다.

만약에, [종가, 시가, 고가] 3가지 feature를 모두 본다면, 3이 될 것 입니다.

#### 1-1. return_sequences=False 인 경우

In [4]:
lstm = LSTM(20)

In [5]:
output_a = lstm(x)

In [6]:
output_a[:5]

<tf.Tensor: shape=(5, 20), dtype=float32, numpy=
array([[ 0.1517384 ,  0.06677328,  0.12768444, -0.13893387,  0.06300796,
         0.03913334, -0.14013042, -0.10510724, -0.17377186, -0.05460759,
        -0.11427589,  0.16377658, -0.17032014, -0.13667141,  0.01065405,
         0.0583236 , -0.03375912,  0.02317264, -0.13976483, -0.04736975],
       [ 0.14443356,  0.0636319 ,  0.11919659, -0.1263673 ,  0.05633989,
         0.03493879, -0.13227922, -0.09603471, -0.15708697, -0.04982033,
        -0.10312618,  0.14518668, -0.15773733, -0.12592201,  0.00759121,
         0.05118459, -0.02929701,  0.01830531, -0.12659898, -0.03940209],
       [ 0.14678498,  0.06531915,  0.12166052, -0.12388945,  0.05478437,
         0.03582085, -0.13263544, -0.09866769, -0.15393342, -0.0502392 ,
        -0.10422894,  0.14497423, -0.15639874, -0.12515733,  0.00749062,
         0.0484067 , -0.03082696,  0.01784937, -0.12401483, -0.04019998],
       [ 0.12644845,  0.04411152,  0.0711825 , -0.09347108,  0.04206496,

In [7]:
output_a.shape

TensorShape([32, 20])

**결과 해석**

output_a의 shape이 (32, 20)으로 출력됨을 확인할 수 있습니다.
shape가 (32, 20)의 32는 batch의 갯수, 20은 LSTM에서 지정한 unit 수입니다.

#### 1-2. return_sequences=True 인 경우

In [8]:
lstm = LSTM(20, return_sequences=True)

In [9]:
output_b = lstm(x)

In [10]:
output_b[:5]

<tf.Tensor: shape=(5, 25, 20), dtype=float32, numpy=
array([[[-2.31284299e-03, -5.96597837e-03, -1.32914316e-02, ...,
         -8.30985047e-03, -2.95070428e-02, -7.34834140e-03],
        [-3.12436349e-03, -2.89738155e-03, -1.02427853e-02, ...,
         -9.42260120e-03, -2.62237620e-02, -8.46969243e-03],
        [-4.64760698e-03, -3.94235225e-03, -1.42163578e-02, ...,
         -1.36878109e-02, -3.89235318e-02, -1.21558951e-02],
        ...,
        [-3.96395475e-02,  3.69881862e-03, -2.73923557e-02, ...,
         -7.29250237e-02, -1.91806704e-01, -7.48594329e-02],
        [-3.95453051e-02,  3.09722600e-05, -2.97634061e-02, ...,
         -7.12744594e-02, -1.99186832e-01, -7.11652711e-02],
        [-4.15909030e-02, -8.33782088e-03, -4.37438898e-02, ...,
         -7.70458952e-02, -2.35217780e-01, -7.58226737e-02]],

       [[-1.71032373e-03, -4.32417961e-03, -9.84261371e-03, ...,
         -6.13591960e-03, -2.14421209e-02, -5.44094620e-03],
        [-7.66024971e-03, -1.75753534e-02, -3.7957

In [11]:
output_b.shape

TensorShape([32, 25, 20])

**결과 해석**

output_b의 shape이 (32, 25, 20)으로 출력됌을 확인할 수 있습니다.
shape가 (32, 25, 20)의 32는 batch의 갯수, (25, 20)은 LSTM에서 지정한 unit 수입니다.

### 2. LSTM layer의 결과 값을 Dense로 넘겨줄 경우
아래와 같이 output_a는 return_sequence=False 에 대한 결과 값이며,

output_b는 return_sequences=True 에 대한 결과 값입니다.

In [13]:
output_a.shape

TensorShape([32, 20])

In [14]:
dense = Dense(10)

In [15]:
dense(output_a).shape

TensorShape([32, 10])

위의 결과에서도 나타나듯이, LSTM으로부터 넘겨 받은 20개의 unit이 Dense를 거쳐 10개로 변환됩니다.

#### 2-2. return_sequence=True 를 Dense에 넘겨줄 경우

In [16]:
output_b.shape

TensorShape([32, 25, 20])

In [17]:
dense = Dense(10)

In [18]:
dense(output_b).shape

TensorShape([32, 25, 10])

이번에도 마찬가지로, LSTM으로부터 넘겨 받은 20개의 unit이 Dense를 거쳐 10개로 변환됩니다.

단, shape는 이전 케이스와는 다르게 모든 sequence에 대한 유닛 20개를 10개로 변환된 것을 확인할 수 있습니다.

#### 2-3. TimeDistributed layer 활용
TimeDistributed layer는 return_sequences=True 인 경우, sequence로 받은 데이터에 대하여 처리할 수 있지만, 사실상 Dense를 써주면 동작은 동일하게 나타납니다.

유의해야할 점은, return_sequences=False로 받은 값은 2차원이기 때문에 TimeDistributed에 넘겨줄 수 없습니다.

##### Dense를 사용한 경우

In [ ]:
dense = Dense(10)

In [19]:
dense(output_b).shape

TensorShape([32, 25, 10])

##### TimeDistributed를 사용한 경우

In [20]:
time = TimeDistributed(Dense(10))

In [21]:
time(output_b).shape

TensorShape([32, 25, 10])

### 3. cell state와 hidden layer
LSTM layer에서 결과 값에 대한 hidden layer 값과, cell state를 받아볼 수 있습니다.

In [22]:
lstm = LSTM(20, return_sequences=True, return_state=True)

In [23]:
output_c, _hidden, _state = lstm(x)

In [24]:
output_c.shape

TensorShape([32, 25, 20])

In [25]:
_hidden.shape

TensorShape([32, 20])

In [26]:
_state.shape

TensorShape([32, 20])